In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
import argparse
import torch
from torch import nn
import matplotlib.pyplot as plt
from stu import STU
import time
import random

Unable to import FlashFFTConv: No module named 'flashfftconv'. Falling back to PyTorch implementation.


In [3]:
from lds import LDS

In [6]:
layer_i = 0
state_dim = 10000
batch_size = 2
epochs = 4000
seq_len = 512
kx = 5
lr = 0.0001

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the layer i weights
stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_500m_param_full.pt", map_location=device)
stu_layer_full.to(device)

# Initialize LDS model
lds = LDS(state_dim, 768, 768, kx).to(device)
optimizer = torch.optim.Adam(lds.parameters(), lr=lr)

# Training
lds_loss_values = []

best_loss = float('inf')

/tmp/ipykernel_2185115/955826039.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_500m_param_full.pt", ma

In [7]:
for epoch in range(epochs):
    inputs = torch.randn(batch_size, seq_len, 768).to(device).to(torch.bfloat16)
    stu_outputs = stu_layer_full(inputs).to(device)

    optimizer.zero_grad()
    loss = lds.compute_loss(inputs.to(stu_outputs.dtype), stu_outputs)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(lds.parameters(), max_norm=1)
    lds_loss_values.append(loss.item())
    optimizer.step()

    with torch.no_grad():
        lds.A.data.clamp_(max=1, min=-1)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.10473068803548813
Epoch 10, Loss: 0.09524866193532944
Epoch 20, Loss: 0.08576725423336029
Epoch 30, Loss: 0.07761033624410629
Epoch 40, Loss: 0.07039359211921692
Epoch 50, Loss: 0.06352762877941132
Epoch 60, Loss: 0.05657444894313812
Epoch 70, Loss: 0.050816860049963
Epoch 80, Loss: 0.04517318308353424
Epoch 90, Loss: 0.04052567109465599
Epoch 100, Loss: 0.036130230873823166
Epoch 110, Loss: 0.03232709318399429
Epoch 120, Loss: 0.028550341725349426
Epoch 130, Loss: 0.025577977299690247
Epoch 140, Loss: 0.022773314267396927
Epoch 150, Loss: 0.02022547274827957
Epoch 160, Loss: 0.01809532567858696
Epoch 170, Loss: 0.016186002641916275
Epoch 180, Loss: 0.014503531157970428
Epoch 190, Loss: 0.012922229245305061
Epoch 200, Loss: 0.011624063365161419
Epoch 210, Loss: 0.010369960218667984
Epoch 220, Loss: 0.009369440376758575
Epoch 230, Loss: 0.008439574390649796
Epoch 240, Loss: 0.007592191454023123
Epoch 250, Loss: 0.0069134896621108055
Epoch 260, Loss: 0.006281384732574224